In [ ]:
import findspark
findspark.init()
findspark.find()

'/Users/choyubin/Documents/spark/spark'

In [2]:
from pyspark.sql import SparkSession

#Create a Spark session
spark = SparkSession.builder.appName("Read Parquet").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 20:24:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df = spark.read.format("csv").option("header", "true").load("../../../Downloads/Spark-The-Definitive-Guide-master/data/retail-data/all/*.csv").coalesce(5)

In [10]:
df.cache()
df.createOrReplaceTempView("dfTable")

- 집계를 수행하려면 키나 그룹을 지정하고 하나 이상의 컬럼을 변환하는 방법을 지정하는 집계 함수를 사용
- 집계 함수는 여러 입력값이 주어지면 그룹별로 결과를 생성
# 7.1 집계 함수
- 모든 집계는 6장에서 사용한 DataFrame의 .stat 속성을 이용하는 특별한 경우를 제외한다면 함수를 사용
- 집계 함수는 org.apache.spark.sql.functions 패키지

In [11]:
df.count() == 541909

True

In [12]:
df.show(5)


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



## 7.1.1 count
- 두 가지 방식으로 사용
    1. count 함수에 특정 컬럼을 지정하는 방식
    2. count(*)나 count(1)을 사용하는 방식

In [18]:
from pyspark.sql.functions import count

In [19]:
df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



## 7.1.2 countDistinct

- 전체 레코드 수가 아닌 고유 레코드 수를 구할 때 사용
- 개별 컬럼을 처리하는 데 더 적합

In [21]:
from pyspark.sql.functions import countDistinct

In [22]:
df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



## 7.1.3 approx_count_distinct

- 근사치
    - 정확한 고유 개수가 무의미한 경우
- 최대 추정 오류율이라는 한 가지 파라미터를 더 사용
- countDistinct 함수보다 더 빠르게 결과 반환
- 대규모 데이터셋을 사용할 때 성능이 훨씬 좋아짐

In [23]:
from pyspark.sql.functions import approx_count_distinct

In [24]:
df.select(approx_count_distinct("StockCode",0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



## 7.1.4 first와 last

- DataFrame의 첫 번째 값이나 마지막 값을 얻을 때 사용
- 값이 아닌 로우를 기반으로 동작

In [25]:
from pyspark.sql.functions import first, last

In [26]:
df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



## 7.1.5 min과 max

- DataFrame에서 최솟값과 최댓값을 추출

In [27]:
from pyspark.sql.functions import min, max

In [28]:
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|           -1|          992|
+-------------+-------------+



## 7.1.6 sum

- DataFrame에서 특정 컬럼의 모든 값을 합산

In [29]:
from pyspark.sql.functions import sum

In [30]:
df.select(sum("Quantity")).show()

+-------------+
|sum(Quantity)|
+-------------+
|    5176450.0|
+-------------+



## 7.1.7 sumDistinct

- 고윳값을 합산

## 7.1.8 avg

- sum 함수의 결과를 count 함수의 결과로 나누어 평균값을 구할 수 있지만, 스파크의 avg 함수나 mean 함수를 사용하면 평균값을 더 쉽게 구할 수 있음

In [31]:
from pyspark.sql.functions import sum, count, avg, expr

In [32]:
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases")).selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



## 7.1.9 분산과 표준편차

- 평균 주변에 데이터가 분포된 정도를 측정하는 방법
- 분산은 평균과의 차이를 제곱한 결과의 평균
- 표준편차는 분산의 제곱근
- 스파크는 표본표준편차 뿐만 아니라 모표준편차 방식도 지원하므로 주의가 필요
- variance 함수나 stddev 함수를 사용한다면 기본적으로 표본표준분산과 표본표준편차 공식을 이용
- var_pop 함수나 stddev_pop 함수를 사용한다면 모표준분산이나 모표준편차 방식을 사용

In [33]:
from pyspark.sql.functions import var_pop, stddev_pop, var_samp,stddev_samp

In [34]:
df.select(var_pop("Quantity"), var_samp("Quantity"), stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660918| 47559.39140929887|  218.08095663447824|   218.08115785023443|
+-----------------+------------------+--------------------+---------------------+



## 7.1.10 비대칭도와 첨도

- 데이터의 변곡점을 측정하는 방법
- 비대칭도 : 데이터 평균의 비대칭 정도 측정
- 첨도 : 데이터 끝 부분을 측정
- 확률변수의 확률분포로 데이터를 모델링할 때 특히 중요

## 7.1.11 공분산과 상관관계

- 지금까지는 단일 컬럼 집계 함수였으나 이제 두 컬럼값 사이의 영향도를 비교하는 함수
- cov, corr : 공분산, 상관관계
- 공분산은 데이터 입력값에 따라 다른 범위를 가짐
- 상관관계는 피어슨 상관계수를 측정하며 -1과 1 사이의 값을 가짐
- var 함수처럼 표본공분산 방식이나 모공분산 방식으로 공분산을 계산할 수도 있음

## 7.1.12 복합 데이터 타입의 집계

- 스파크는 수식을 이용한 집계뿐만 아니라 복합 데이터 타입을 사용해 집계를 수행할 수 있음
    - ex. 특정 컬럼의 값을 리스트로 수집하거나 셋 데이터 타입으로 고윳값만 수집할 수 있음
- 수집된 데이터는 처리 파이프라인에서 다양한 프로그래밍 방식으로 다루거나 사용자 정의 함수를 사용해 전체 데이터에 접근 가능

In [35]:
from pyspark.sql.functions import collect_set, collect_list

In [36]:
df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



# 7.2 그룹화

- 그룹 기반의 집계를 수행하는 경우가 많음
- 단일 컬럼의 데이터를 그룹화하고 해당 그룹의 다른 여러 컬럼을 사용해서 계산하기 위해 카테고리형 데이터를 사용
    - ex. 고유한 송장번호(InvoiceNo)를 기준으로 그룹을 만들고 그룹별 물품 수를 카운트. 이 연산은 또 다른 DataFrame을 반환하며 지연 처리 방식으로 수행됨
- 그룹화 작업은 하나 이상의 컬럼을 그룹화하고 집계 연산을 수행하는 두 단계로 이뤄짐
    - 첫 번째 단계에서는 RelationalGroupedDataset이 반환됨
    - 두 번째 단계에서는 DataFrame이 반환됨
- 그룹의 기준이 되는 컬럼을 여러 개 지정할 수 있음

In [37]:
df.groupBy("InvoiceNo", "CustomerID").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerID|count|
+---------+----------+-----+
|   536395|     13767|   14|
|   536609|     17850|   16|
|   536785|     15061|    6|
|   537205|     13034|   11|
|   537829|     15498|   19|
|   538513|     15454|   25|
|  C538642|     12476|    2|
|   538808|     14057|   43|
|   538830|     14298|    5|
|   539374|     14769|   22|
|   539688|     12678|    1|
|  C539956|     12980|    1|
|   539988|     18116|   89|
|  C540384|     14961|    3|
|   540498|     17243|   10|
|   541595|     14606|   20|
|   541858|     16983|   14|
|  C542218|     14606|    3|
|  C542643|     13615|    1|
|   542788|     16570|   15|
+---------+----------+-----+
only showing top 20 rows



## 7.2.1 표현식을 이용한 그룹화

- agg 메서드 : 여러 집계 처리를 한 번에 지정할 수 있으며, 집계에 표현식을 사용할 수 있음. 또한 트랜스포메이션이 완료된 컬럼에 alias 메서드를 사용할 수 있음

In [38]:
from pyspark.sql.functions import count

In [39]:
df.groupBy("InvoiceNo").agg(count("Quantity").alias("quan"), expr("count(Quantity)")).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|  C542604|   8|              8|
+---------+----+---------------+
only showing top 20 rows



## 7.2.2 맵을 이용한 그룹화

- 컬럼을 키로, 수행할 집계 함수의 문자열을 값으로 하는 맵 타입을 사용해 트랜스포메이션을 정의할 수 있음
- 수행할 집계 함수를 한 줄로 작성하면 여러 컬럼명을 재사용할 수 있음

In [40]:
df. groupBy("InvoiceNo").agg(expr("avg(Quantity)"), expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

# 7.3 윈도우 함수

- 윈도우 함수는 데이터의 특정 ‘윈도우’를 대상으로 고유의 집계 연산을 수행
- 데이터의 ‘윈도우’는 현재 데이터에 대한 참조를 사용해 정의
- 윈도우 명세는 함수에 전달될 로우를 결정
- 표준 group-by 함수와 유사해보일 수 있으나 차이점이 있음
    - group-by 함수를 사용하면 모든 로우 레코드가 단일 그룹으로만 이동
    - 윈도우 함수는 프레임에 입력되는 모든 로우에 대해 결괏값을 계산. 프레임은 로우 그룹 기반의 테이블을 의미. 각 로우는 하나 이상의 프레임에 할당될 수 있음
- 가장 흔하게 사용되는 방법 중 하나는 하루를 나타내는 값의 롤링 평균을 구하는 것.
    - 이 작업을 수행하려면 개별 로우가 7개의 다른 프레임으로 구성되어야 함
    - 스파크는 다음 세 가지 종류의 윈도우 함수를 지원
        1. 랭크 함수(ranking function)
        2. 분석 함수(analytic function)
        3. 집계 함수(aggregate function)

In [42]:
from pyspark.sql.functions import col, to_date

In [43]:
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

In [44]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

In [46]:
windowSpec = Window.partitionBy("CustomerId", "date").orderBy(desc("Quantity")).rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [47]:
from pyspark.sql.functions import max

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [49]:
from pyspark.sql.functions import dense_rank, rank

purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [52]:
from pyspark.sql.functions import col

In [54]:
dfWithDate.where("CustomerId IS NOT NULL").orderBy("customerId")\
.select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

23/03/15 22:48:50 ERROR Executor: Exception in task 3.0 in stage 45.0 (TID 89)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '9/25/2011 10:30' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1084)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:176)
	at org.apache.spar

Py4JJavaError: An error occurred while calling o217.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 45.0 failed 1 times, most recent failure: Lost task 1.0 in stage 45.0 (TID 87) (192.168.0.2 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '3/14/2011 13:04' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1084)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.Abst

# 7.4 그룹화 셋
- 여러 그룹에 걸쳐 집계
- GROUPING SETS 구문은 SQL에서만 사용 가능
- DataFrame에서 동일한 연산을 수행하려면 rollup 메서드와 cube 메서드 사용

In [55]:
dfNoNull = dfWithDate.na.drop()

In [56]:
dfNoNull.createOrReplaceTempView("dfNoNull")

## 7.4.1 롤업
- 다양한 컬럼을 그룹화 키로 설정하면 그룹화 키로 설정된 조합뿐만 아니라 데이터셋에서 볼 수 있는 실제 조합을 모두 살펴볼 수 있음
- 롤업은 group-by 스타일의 다양한 연산을 수행할 수 있는 다차원 집계 기능

In [60]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity")).selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity").orderBy("Date")
rolledUpDF.show()

23/03/15 23:07:06 ERROR Executor: Exception in task 2.0 in stage 48.0 (TID 103)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '6/23/2011 15:35' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1084)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:176)
	at org.apache.spa

Py4JJavaError: An error occurred while calling o301.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 48.0 failed 1 times, most recent failure: Lost task 2.0 in stage 48.0 (TID 103) (192.168.0.2 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '6/23/2011 15:35' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1084)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.Abs

## 7.4.2 큐브
- 큐브는 롤업을 고차원적으로 사용할 수 있게 해줌
- 요소들을 계층적으로 다루는 대신 모든 차원에 대해 동일한 작업을 수행

In [61]:
from pyspark.sql.functions import sum

In [62]:
dfNoNull.cube("Date", "Country").agg(sum(col("Quantity"))).select("Date", "Country", "sum(Quantity)").orderBy("Date").show()

23/03/15 23:08:40 ERROR Executor: Exception in task 1.0 in stage 49.0 (TID 107)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '3/14/2011 13:04' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1084)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:176)
	at org.apache.spa

Py4JJavaError: An error occurred while calling o325.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 49.0 failed 1 times, most recent failure: Lost task 3.0 in stage 49.0 (TID 109) (192.168.0.2 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '9/23/2011 17:02' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1084)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.Abs

## 7.4.3 그룹화 메타데이터
- grouping_id : 결과 데이터셋의 집계 수준을 명시하는 컬럼을 제공

## 7.4.4 피벗
- 로우를 컬럼으로 변환

In [63]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

In [66]:
pivoted.where("date > '2011-12-05'").select("date", "USA_sum(Quantity)").show()

AnalysisException: Column '`USA_sum(Quantity)`' does not exist. Did you mean one of the following? [date];
'Project [date#4214, 'USA_sum(Quantity)]
+- Filter (date#4214 > cast(2011-12-05 as date))
   +- Project [date#4214]
      +- Aggregate [date#4214], [date#4214]
         +- Aggregate [date#4214, Country#24], [date#4214, Country#24]
            +- Project [InvoiceNo#17, StockCode#18, Description#19, Quantity#20, InvoiceDate#21, UnitPrice#22, CustomerID#23, Country#24, to_date(InvoiceDate#21, Some(MM/d/yyyy H:mm), Some(Asia/Seoul)) AS date#4214]
               +- Repartition 5, false
                  +- Relation [InvoiceNo#17,StockCode#18,Description#19,Quantity#20,InvoiceDate#21,UnitPrice#22,CustomerID#23,Country#24] csv


# 7.5 사용자 정의 집계 함수
- UDAF는 직접 제작한 함수나 비즈니스 규칙에 기반을 둔 자체 집계 함수를 정의하는 방법
- UDAF를 사용해서 입력 데이터 그룹에 직접 개발한 연산을 수행할 수 있음
- 스파크는 입력 데이터의 모든 그룹의 중간 결과를 단일 AggregationBuffer에 저장해 관리
- UDAF를 생성하려면 기본 클래스인 UserDefinedAggregateFunction을 상속받음
- 그리고 다음과 같은 메서드를 정의
    - inputSchema
    - bufferSchema
    - dataType
    - deterministic
    - initialize
    - update
    - merge
    - evaluate
- UDAF는 현재 스칼라와 자바로만 사용 가능    